In [ ]:
# load model with random weights
resnext = models.resnext101_32x8d(pretrained=True)

for param in resnext.parameters():
    param.requires_grad = False
print(resnext)

In [ ]:
# change the output layer
num_classes=10
num_ftrs = resnext.fc.in_features 
resnext.fc= nn.Linear(num_ftrs, num_classes)
resnext.to(device);
print(resnext)

In [ ]:
#Model Training
lr = 0.0001
loss_func = nn.CrossEntropyLoss(weight=weights_tensor,reduction="sum")
opt = optim.Adam(resnext.parameters(), lr=lr)
lr_scheduler = StepLR(opt,step_size=7, gamma=0.1)

params_train={
 "num_epochs": 10,
 "optimizer": opt,
 "loss_func": loss_func,
 "train_dl": train_loader,
 "val_dl": valid_loader,
 "sanity_check": False,
 "lr_scheduler": lr_scheduler,
 "path2weights": "resnext.pt",
}

# train and validate the model
resnext,loss_hist,metric_hist=train_val(device,resnext,params_train)

In [ ]:
#Model Evaluation
resnext.eval();
y_out,y_gt=deploy_model(resnext,valid_dataset,device=device)

y_pred = np.argmax(y_out,axis=1)
acc=accuracy_score(y_pred,y_gt)
print(acc)